In [31]:
# -*- coding: utf-8 -*-
"""
Created on Thu Feb 23 07:37:51 2017

@author: Wu
"""

import os
import math
import pandas as pd
import numpy as np
os.chdir('c:/coding/228')

# estimate 1 : using SDR to estimate 228 death count
(SDR means "Sexualized Death Ratio")

In [32]:
data=pd.read_csv('data.txt',sep='\t')
print data.head(12)

    year  month  male  female
0   1947      1  5540    4740
1   1947      2  4925    4259
2   1947      3  5640    4678
3   1947      4  5864    4938
4   1947      5  5332    4661
5   1947      6  4655    4194
6   1947      7  4834    4519
7   1947      8  4838    4444
8   1947      9  4628    4403
9   1947     10  4725    4452
10  1947     11  4535    4064
11  1947     12  4879    4449


In [33]:
normalData=data.loc[(data['year']!=1947) & (data['month'].isin([3,4,5])) ,:]
normalData.loc[:,'SDR']=normalData.apply(lambda row:float(row['male'])/row['female'],axis=1)
normalSDR=normalData.groupby('month').apply(lambda gData:gData['SDR'].mean())
normalSDR_indexed=pd.DataFrame({u'1949~1951年台灣地區死亡性別比例(SDR)':normalSDR})
print normalSDR_indexed

       1949~1951年台灣地區死亡性別比例(SDR)
month                                      
3                                  1.100507
4                                  1.100636
5                                  1.050343


In [34]:
inciData=data.loc[(data['year']==1947) & (data['month'].isin([3,4,5])) ,:]
inciData.loc[:,'SDR']=normalSDR.tolist()
inciData.loc[:,'maleEstimate']=inciData.apply(lambda row:row['female']*row['SDR'],axis=1)
inciData.loc[:,'228Estimate']=inciData.apply(lambda row:row['male']-row['maleEstimate'],axis=1)
estimate1=inciData['228Estimate'].sum()
print inciData
print 'estimate1: {}'.format(math.ceil(estimate1))


   year  month  male  female       SDR  maleEstimate  228Estimate
2  1947      3  5640    4678  1.100507   5148.173311   491.826689
3  1947      4  5864    4938  1.100636   5434.939615   429.060385
4  1947      5  5332    4661  1.050343   4895.651019   436.348981
estimate1: 1358.0


# estimate 2: using death date in compensation data to modify estimate 1

In [38]:
execfile('cleanCompenData.py')
compen=pd.read_csv('cleanCompenData.txt',sep='\t')
compen['deathDate']=data['deathDate'].fillna('')
compen['deathDate']=compen['deathDate'].astype('str')
compen['monthSince1947']=compen['deathDate'].apply(lambda x:int(x[4:6])+(int(x[:4])-1947)*12 if x!='' else None)
print compen.loc[:,['name','deathDate','monthSince1947']].head(20)

      name deathDate  monthSince1947
0   尤萬居  19470306             3.0
1   方振東  19470304             3.0
2   王天炳  19470309             3.0
3   王天富  19470321             3.0
4     王木  19470315             3.0
5   王平水  19470306             3.0
6   王名朝  19470405             4.0
7   王有福  19470417             4.0
8   王育霖  19470314             3.0
9   王來湖                       NaN
10  王東朝  19470301             3.0
11  王金火  19470308             3.0
12  王金星  19470314             3.0
13  王金振                       NaN
14  王金塗  19470301             3.0
15  王金樹                       NaN
16  王金興  19470308             3.0
17  王垢復  19470307             3.0
18  王炳煌  19470303             3.0
19  王國家  19470228             2.0


In [39]:
monDeath=compen['monthSince1947'].value_counts(dropna=False)

def combineMonth(x):
    if x==2:
        return u'1947年二月'
    elif x>2 and x<6:
        return u'1947年三到五月'
    elif np.isnan(x) or x is None:
        return u'無死亡時間'
    else:
        return u'1947年六月以後'
        
monDeath=monDeath.groupby(combineMonth).sum()
monDeath=monDeath.reindex_axis([u'1947年二月',u'1947年三到五月',u'1947年六月以後',u'無死亡時間'])
monDeath_indexed=pd.DataFrame({u'受補償受難者死亡月份':monDeath})
print monDeath_indexed

                受補償受難者死亡月份
1947年二月                        19
1947年三到五月                   556
1947年六月以後                     8
無死亡時間                       205


In [40]:
monDeath=monDeath.drop(u'無死亡時間')
monDeath=pd.DataFrame({u'死亡人數':monDeath,u'所佔比例':monDeath/monDeath.sum()})
print monDeath

                所佔比例  死亡人數
1947年二月      0.032590        19
1947年三到五月  0.953688       556
1947年六月以後  0.013722         8


In [41]:
estimate2=estimate1/monDeath.loc[u'1947年三到五月',u'所佔比例']
print 'estimate2: {}'.format(math.ceil(estimate2))


estimate2: 1424.0
